In [1]:
# Adapting "2025-02-13 BC14 and TCAV3 emittance scans.ipynb" and "2025-03-11 Importing beams from Jingyi.ipynb"
from UTILITY_quickstart import *

importedDefaultSettings = loadConfig("setLattice_configs/2024-10-22_oneBunch_baseline.yml")

In [2]:
csrTF = True


bunchCount = importedDefaultSettings["bunchCount"]

tao = initializeTao(
    inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"],
    
    csrTF = csrTF,
    numMacroParticles=1e4,
    scratchPath = "/tmp",
    randomizeFileNames = True
)

setLattice(tao, **importedDefaultSettings)
launchTwissCorrection(tao)


trackStart = "ENDBC14E"
trackEnd = "PR15944"


tao.cmd(f"set ele TCY15280 VOLTAGE = 20e6")
tao.cmd(f"set ele TCY15280 PHI0 = {0/360.0}")

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR on
Overwriting lattice with setLattice() defaults
No defaults file provided to setLattice(). Using setLattice_configs/defaults.yml
Number of macro particles = 10000.0
Beam created, written to /tmp/beams/activeBeamFile_239865241995219094.h5, and reinit to tao
No evalElement provided. Assuming golden lattice PR10571
Optimization Results:
Optimal Parameters: [28.77872199 13.3970811  23.51278494 16.8307956 ]
Objective Function Value at Optimal Parameters: 9.505958617749993e-12
Number of Iterations: 260
Converged: True


[]

## Working on parsing and importing Yunhai's format

In [3]:
trackBeam(tao, trackEnd = "ENDBC14E", **importedDefaultSettings)
P = getBeamAtElement(tao, "ENDBC14E")

In [4]:
print(f"""x, y, z: {P["sigma_x"]}, {P["sigma_y"]}, {P["sigma_z"]}""")

print(f"""px, py, pz: {P["sigma_px"]}, {P["sigma_py"]}, {P["sigma_pz"]}""")

print(f"""xp, yp, rel_pz: {P["sigma_xp"]}, {P["sigma_yp"]}, {P["sigma_pz"]/P["mean_pz"]}""")

print(f"""energy: {P["mean_energy"]}""")

print(f"""mean z, t:  {P["mean_z"]}, {P["mean_t"]}""")

print(P["sigma_t"] * 3e8)

#Don't trust... but seems that (t - <t>) c = -z 
print( (P["t"] - P["mean_t"]) * 3e8 + P["z"] ) 

x, y, z: 0.00011662824497923542, 5.9994865850397885e-05, 3.7010669610774685e-05
px, py, pz: 79027.55614025336, 40981.66829042848, 61691353.65571428
xp, yp, rel_pz: 1.7797030582425663e-05, 9.249301725835544e-06, 0.013922499197789345
energy: 4431054597.976368
mean z, t:  -5.036257813970496e-14, 1.3915374152912633e-06
3.70362915641874e-05
[-4.26612359e-09  1.00165630e-08  4.53523510e-08 ... -4.52054616e-09
  5.38384562e-09 -1.59845239e-08]


In [5]:
print(P["status"])
print(P["weight"])
print(P["species"])

[1 1 1 ... 1 1 1]
[1.6e-13 1.6e-13 1.6e-13 ... 1.6e-13 1.6e-13 1.6e-13]
electron


In [6]:
import json
with open('/Users/nmajik/testExport.json', 'r') as f:
    d = json.load(f)

In [7]:
P = ParticleGroup(data = d)

In [8]:
print(f"""x, y, z: {P["sigma_x"]}, {P["sigma_y"]}, {P["sigma_z"]}""")

print(f"""px, py, pz: {P["sigma_px"]}, {P["sigma_py"]}, {P["sigma_pz"]}""")

print(f"""xp, yp, rel_pz: {P["sigma_xp"]}, {P["sigma_yp"]}, {P["sigma_pz"]/P["mean_pz"]}""")

print(f"""energy: {P["mean_energy"]}""")

print(f"""mean z, t:  {P["mean_z"]}, {P["mean_t"]}""")

print(P["sigma_t"] * 3e8)

#Don't trust... but seems that (t - <t>) c = -z 
print( (P["t"] - P["mean_t"]) * 3e8 + P["z"] ) 

x, y, z: 0.00011872496398473486, 5.2505276141230385e-05, 5.2505276141230385e-05
px, py, pz: 80952.98596091988, 41672.83820646465, 62622838.011790514
xp, yp, rel_pz: 1.8243814098060392e-05, 9.407436498910284e-06, 0.014137425749875147
energy: 4429578592.238009
mean z, t:  -1.347344162162338e-07, 1.3915374536556716e-06
3.3088955964281794e-05
[ 7.93779497e-05  3.76446036e-05  2.09166265e-06 ...  3.71527030e-05
 -2.36504701e-05  8.02975409e-05]


In [9]:
PReference = getBeamAtElement(tao, "ENDBC14E")

In [10]:
fileSuffix = "reference"
PStart = PReference

#Figures for starting beam
fig = plotMod(PStart, 'x', 'xp', bins=300, xlim = (-0.0005, 0.0005), ylim = (-100e-6, 50e-6) )
fig.savefig(f"/Users/nmajik/{fileSuffix}_xxp.png")
fig = plotMod(PStart, 'delta_t', 'pz', bins=300, xlim = (-200e-15, 200e-15), ylim = (4.3e9, 4.6e9))
fig.savefig(f"/Users/nmajik/{fileSuffix}_tpz.png")




#Track. Then plot and dump final beam

makeBeamActiveBeamFile( PStart, tao = tao )
trackBeam(tao, trackStart = trackStart, trackEnd = trackEnd, **importedDefaultSettings)


P = getBeamAtElement(tao, trackEnd)

fig = plotMod(P, 'x', 'y', bins=300, xlim = (-0.5e-3, 1e-3), ylim = (-1e-3,2e-3))


fig.savefig(f"/Users/nmajik/{fileSuffix}_streaked.png")

P.write(f"/Users/nmajik/{fileSuffix}_streaked.h5")


plt.clf()

<Figure size 640x480 with 0 Axes>

## Loop to make images

In [11]:
fileSuffixes = [
    "chirp0_0.1Dcsr.json",
    "chirp0_005.1Dcsr.json",
    "chirp0_01.1Dcsr.json",
    "chirp0_015.1Dcsr.json",
    "chirp0_02.1Dcsr.json",
    "chirp0_05.1Dcsr.json",

    "chirp0_0.2Dcsr.json",
    "chirp0_005.2Dcsr.json",
    "chirp0_01.2Dcsr.json",
    "chirp0_015.2Dcsr.json",
    "chirp0_02.2Dcsr.json",
    "chirp0_05.2Dcsr.json",
]

for fileSuffix in fileSuffixes:

    with open(f'/Users/nmajik/Downloads/LEGO_1D_2D/{fileSuffix}', 'r') as f:
        d = json.load(f)

    
    PStart = ParticleGroup(data = d)

    #Figures for starting beam
    fig = plotMod(PStart, 'x', 'xp', bins=300, xlim = (-0.0005, 0.0005), ylim = (-100e-6, 50e-6) )
    fig.savefig(f"/Users/nmajik/{fileSuffix}_xxp.png")
    fig = plotMod(PStart, 'delta_t', 'pz', bins=300, xlim = (-200e-15, 200e-15), ylim = (4.3e9, 4.6e9))
    fig.savefig(f"/Users/nmajik/{fileSuffix}_tpz.png")




    #Track. Then plot and dump final beam
    
    makeBeamActiveBeamFile( PStart, tao = tao )
    trackBeam(tao, trackStart = trackStart, trackEnd = trackEnd, **importedDefaultSettings)


    P = getBeamAtElement(tao, trackEnd)

    fig = plotMod(P, 'x', 'y', bins=300, xlim = (-0.5e-3, 1e-3), ylim = (-2e-3,2e-3))

    
    fig.savefig(f"/Users/nmajik/{fileSuffix}_streaked.png")

    P.write(f"/Users/nmajik/{fileSuffix}_streaked.h5")

    
    plt.clf()

    print(f""" {fileSuffix}, {P["sigma_z"]}, {P["mean_x"]}, {P["sigma_x"]}""")
 

 chirp0_0.1Dcsr.json, 2.383748254493385e-05, 0.0002634650988007763, 0.0002806055913115039
 chirp0_005.1Dcsr.json, 2.6097515769549958e-05, 0.00019601815051305602, 0.0001939935011409852
 chirp0_01.1Dcsr.json, 2.8396146687204098e-05, 0.00015928194033150214, 0.00015315054984232023
 chirp0_015.1Dcsr.json, 3.072566838616945e-05, 0.00013658293648854202, 0.00013236638650048532
 chirp0_02.1Dcsr.json, 3.3079781026537226e-05, 0.00012112958305934318, 0.00012085203991108827
 chirp0_05.1Dcsr.json, 4.751480495693093e-05, 7.970001537963314e-05, 0.00010188794158000298
 chirp0_0.2Dcsr.json, 2.4375605741747868e-05, 0.0003360137709624563, 0.0003245530302676438
 chirp0_005.2Dcsr.json, 2.6537972562744715e-05, 0.00024908835824689635, 0.00022737528729237024
 chirp0_01.2Dcsr.json, 2.8769580772848472e-05, 0.0002016094394410521, 0.00017900105478531092
 chirp0_015.2Dcsr.json, 3.105086607297616e-05, 0.00017093692757511465, 0.00015151694856240286
 chirp0_02.2Dcsr.json, 3.336479882638767e-05, 0.00014934981427679831,